# Encryption of the Uncapitalized Text

Decap time = 1.5 hours. Find count time = 1.5 hours. The encoder seems to want 3 hours right now.

In [2]:
#libraries
import string
import encode
import numpy as np

#load text
with open("MrZoat_WithThisRingS01_Gould.txt") as f:
    text = f.read()

text = text.lower()

low = []
for i in range(26):
    low.append(string.ascii_lowercase[i])
num = []
for i in range(10):
    num.append(string.digits[i])

# now in the correct order
sim1 = ['\n', ' ', '!', '\"', '$', '%', '&', '\'', '(', ')', '*', '+', ',', '-', '.', '/' ]
sim2 = [':', ';', '<', '>', '?', '[', ']']
sim3 = ['|', '}']
ch = sim1 + num + sim2 + low + sim3 # this is our full list of symbols

In [3]:
# This is the long part
c1_non0, c2_non0, c3_non0, c3n_non0, c4_non0, count4, c4_arr = encode.find_counts_text_enc(text, ch)

In [4]:
counts_enc = encode.byte_mng_enc()
encode.encode_counts_text(text, counts_enc, ch, c2_non0, c3n_non0, c4_non0, c4_arr)

# Text Compression

My assumption is that I already have the capitalization, which I will not deal with here, and the context-based probabilities, which are already loaded. Do to how I store the probabilities, I will ignore the first three characters (already recorded in the model) and work only with probabilities based on the previous three characters.

I need routines to: 
{l_old, h_old, bits_old -> a, b -> h-l+1, h-l+1 / 2^n -> a+ term, b+ term -> l, h -> bin l, bin h -> enc update -> new bin l, new bin h, bits_new -> new l, new h}
 - find the counts based on the context (give context, find counts)
 - find the probabilities and borders based on the counts (give counts and char, find left and right border)
 - store bits and bytes, take bits and add it to the byte list (input array of bools, update encoding)
 - keep track of and update l, h, bits

I can fix the end of file issue by adding 3 bits to the beginning, setting them to 0, then later setting them to encode how many zeros at the end of the text are not intended / don't indicate extra cycles. This prevents from needing to play around with the order of the files.

In [5]:
c2_non0v2 = [list(c2_non0[0]), list(c2_non0[1])]
c3_non0v2 = [list(c3n_non0[0]), list(c3n_non0[1])]
c4_non0v2 = [list(c4_non0[0]), list(c4_non0[1])]

In [6]:
context_start = [ch.index(text[0]), ch.index(text[1]), ch.index(text[2])]

counts = encode.count_static_k(3, ch, c4_arr, [c2_non0v2, c3_non0v2, c4_non0v2])
encoder_new = encode.arith_code_enc_k_fixed(context_start, counts)

for i in text[3:]:
    encoder_new.add_character(i)

whitespace2 = encoder_new.finalize()
encoder_new.byte_encode.save_bits('text_encode') # This will be used to test the size of the compressed text once I run the algorithm. 
                                                 # I need to switch 'wb' to 'ab' for this to actually work as intended.
print(encoder_new.duration_static, whitespace2)

# I also need to add a number somewhere to indicate how many 0s are added to the end due to not being a full byte, and the size of the capitalization encoding,
# since neither this nor the captialization encoding have a definite end (I can add an end to the capitalization only when I know the text size)

0 2


In [7]:
max(c3_non0v2[0])

1426

In [8]:
text_byte_array = bytearray(encoder_new.byte_encode.byte_store)

In [9]:
filename = "arith_enc_test_2"
with open(filename, 'wb') as f:
    f.write(text_byte_array)

In [10]:
len(text_byte_array)

1148570